In [1]:
from __future__ import division
import numpy as np
from glob import glob
from os.path import join, basename
from nipype.interfaces import afni

In [3]:
data_dir = '/scratch/PSB6351_2017/students/salo/data/frstlvl_RTamp/'
mask_dir = '/scratch/PSB6351_2017/ds008_R2.0.0/preproc/anat_masks/'

subjects = sorted([basename(s) for s in glob(join(data_dir, 'sub-*'))])
s_acfs = np.empty((len(subjects), 4))
for i, s in enumerate(subjects):
    mask_file = join(mask_dir, s, 'masks/_subject_id_{0}/wb_warped.nii.gz'.format(s))

    runs_dir = join(data_dir, s, 'modelfit/estimates/')
    run_files = sorted(glob(join(runs_dir, 'run-*/sigmasquareds.nii.gz')))

    r_acfs = np.empty((len(run_files), 4))
    for j, r in enumerate(run_files):
        fwhm = afni.FWHMx(in_file=r, acf=True, detrend=True, automask=True)
        res = fwhm.run()
        r_acfs[j, :] = np.array(res.outputs.acf_param)
    s_acf = np.mean(r_acfs, axis=0)
    s_acfs[i, :] = s_acf
f_acf = np.mean(s_acfs, axis=0)

170329-12:44:02,636 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:++ 3dFWHMx: AFNI version=AFNI_16.3.18 (Dec 22 2016) [64-bit]
170329-12:44:02,639 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:++ Authored by: The Bob
170329-12:44:02,641 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:*+ WARNING: -demed and/or -corder and/or -unif ignored: only 1 input sub-bricks
170329-12:44:02,643 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:++ Number of voxels in automask = 0
170329-12:44:02,644 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:** FATAL ERROR: Automask is too small to process
170329-12:44:02,646 interface INFO:
	 stderr 2017-03-29T12:44:02.635952:** Program compile date = Dec 22 2016


RuntimeError: Command:
3dFWHMx -acf -automask -detrend -input /scratch/PSB6351_2017/students/salo/data/frstlvl_RTamp/sub-01/modelfit/estimates/run-00/sigmasquareds.nii.gz -detprefix sigmasquareds_detrend -out sigmasquareds_subbricks.out > sigmasquareds_fwhmx.out
Standard output:

Standard error:
++ 3dFWHMx: AFNI version=AFNI_16.3.18 (Dec 22 2016) [64-bit]
++ Authored by: The Bob
[7m*+ WARNING:[0m -demed and/or -corder and/or -unif ignored: only 1 input sub-bricks
++ Number of voxels in automask = 0
[7m** FATAL ERROR:[0m Automask is too small to process
** Program compile date = Dec 22 2016
Return code: 1
Interface FWHMx failed to run. 

In [ ]:
print f_acf[:, :-1]

In [ ]:
print('Using 2-sided, NN=2 (edge-connected), p=0.01, a=0.05')
print('3mm min: 14')
print('6mm min: 24')
print('9mm min: 65')
print('12mm min: 111')